# 🎨 Free Stable Diffusion WebUI on Google Colab

**Run AUTOMATIC1111's Stable Diffusion WebUI for free on Google Colab — no local GPU required.**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

---

## Setup (one-time, ~6 minutes total)

1. **Enable GPU**: Runtime → Change runtime type → **T4 GPU** → Save
2. **Run Cell 1** — Install dependencies & clone repos (~3-4 min)
3. **Run Cell 2** — Download Realistic Vision V6 model (~2 min)
4. **Run Cell 3** — Launch WebUI (~1 min)
5. **Click the `trycloudflare.com` URL** that appears → start generating!

### What's included
- **Realistic Vision V6** — photorealistic model (not the weak SD 1.5 base)
- PyTorch + xformers (CUDA 12.1, optimized for T4)
- [ADetailer](https://github.com/Bing-su/adetailer) extension (auto-fixes hands & faces)
- Cloudflare Tunnel (reliable public URL, replaces Gradio's unstable `--share`)
- Pre-configured settings: DPM++ SDE + Karras, 30 steps, CFG 5, Hires fix to 1200×800
- Multi-fallback repo cloning (handles Stability-AI's private repo)

### Known limitations
- Free Colab sessions last ~12 hours, then reset — **save your images!**
- The public URL changes every session
- T4 GPU has 15GB VRAM — keep batch size at 1 for hires images

---

In [ ]:
# ============================================================
# Cell 1: Install dependencies & clone repositories
# Runtime: ~3-4 minutes
# ============================================================

print("=" * 60)
print("  CELL 1: Installing dependencies & cloning repositories")
print("=" * 60)

# ── GPU check ──
print("\n🔍 GPU check:")
!nvidia-smi -L

# ── Install PyTorch (cu121) + xformers ──
print("\n📦 Installing PyTorch + xformers...")
!pip uninstall -y torch torchvision torchaudio
!pip install -q torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q pytorch_lightning einops transformers safetensors accelerate

# ── Clone AUTOMATIC1111 WebUI ──
print("\n📦 Cloning AUTOMATIC1111 WebUI...")
!rm -rf /content/stable-diffusion-webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git /content/stable-diffusion-webui
%cd /content/stable-diffusion-webui

# ── Patch for Python 3.12 compatibility ──
# WebUI pins transformers==4.30.2 → needs tokenizers<0.14
# tokenizers 0.13.x has no cp312 wheels and requires Rust to build.
# Fix: relax the pin so a newer tokenizers (with wheels) is used.
print("\n🔧 Patching requirements for Python 3.12...")
!sed -i 's/transformers==4.30.2/transformers>=4.30.2/' requirements_versions.txt
print("  ✅ transformers version pin relaxed")

# ── Pin numpy < 2 globally ──
# ADetailer → ultralytics → numpy 2.x, which breaks scikit-image (np.float_ removed).
# Using PIP_CONSTRAINT ensures ALL pip installs (including launch.py's internal ones) respect the pin.
print("\n🔧 Pinning numpy < 2 (scikit-image compatibility)...")
!echo "numpy<2" > /tmp/pip-constraints.txt
import os
os.environ['PIP_CONSTRAINT'] = '/tmp/pip-constraints.txt'
!pip install -q "numpy<2"
print("  ✅ numpy < 2 pinned globally via PIP_CONSTRAINT")

# ── Install ADetailer extension (auto-fixes distorted hands & faces) ──
print("\n📦 Installing ADetailer extension...")
!git clone https://github.com/Bing-su/adetailer.git extensions/adetailer
if os.path.isdir('extensions/adetailer'):
    print("  ✅ ADetailer installed (auto-detects and repaints hands/faces)")
else:
    print("  ⚠️  ADetailer install failed — hand/face auto-fix unavailable")

# ── Pre-clone repositories ──
# Stability-AI/stablediffusion is PRIVATE. We need a public fork of the
# SD 2.x codebase (NOT CompVis/stable-diffusion which is SD 1.x and
# missing ldm.modules.midas, causing import errors).
print("\n📦 Pre-cloning required repositories...")
!mkdir -p repositories && rm -rf repositories/*

import subprocess, sys, json
from urllib.request import Request, urlopen

def clone_with_fallbacks(dest, urls, name):
    """Try cloning from a list of URLs. Return the first working URL or None."""
    for url in urls:
        print(f"  Trying: {url}")
        r = subprocess.run(
            ["git", "clone", "--config", "core.filemode=false", url, dest],
            capture_output=True, text=True,
        )
        if r.returncode == 0:
            print(f"  ✅ {name} cloned from {url}")
            return url
        subprocess.run(["rm", "-rf", dest], capture_output=True)
        print(f"  ❌ Failed")
    return None

def find_github_forks(owner, repo, n=10):
    """Return clone URLs of the top-n public forks (by stargazers)."""
    try:
        url = f"https://api.github.com/repos/{owner}/{repo}/forks?sort=stargazers&per_page={n}"
        req = Request(url, headers={"Accept": "application/vnd.github.v3+json",
                                     "User-Agent": "SD-Colab"})
        forks = json.loads(urlopen(req, timeout=15).read())
        urls = [f["clone_url"] for f in forks if not f.get("private")]
        if urls:
            print(f"  🔍 Found {len(urls)} public forks via GitHub API")
        return urls
    except Exception as e:
        print(f"  ⚠️  GitHub fork API: {e}")
        return []

def search_github_repos(query, n=5):
    """Search GitHub for matching repos as a last resort."""
    try:
        url = f"https://api.github.com/search/repositories?q={query}+in:name+fork:true&sort=stars&per_page={n}"
        req = Request(url, headers={"Accept": "application/vnd.github.v3+json",
                                     "User-Agent": "SD-Colab"})
        items = json.loads(urlopen(req, timeout=15).read()).get("items", [])
        urls = [r["clone_url"] for r in items if not r.get("private")]
        if urls:
            print(f"  🔍 Found {len(urls)} repos via GitHub search")
        return urls
    except Exception as e:
        print(f"  ⚠️  GitHub search: {e}")
        return []

sd_dest = "repositories/stable-diffusion-stability-ai"

# Strategy 1: known URLs
print("\n📦 Cloning Stable Diffusion repo...")
sd_url = clone_with_fallbacks(sd_dest, [
    "https://github.com/lancedesk/stablediffusion.git",
    "https://github.com/Stability-AI/stablediffusion.git",
    "https://github.com/Hafiidz/stablediffusion.git",
    "https://github.com/camenduru/stablediffusion.git",
], "Stable Diffusion")

# Strategy 2: GitHub API forks
if not sd_url:
    print("\n  🔍 Trying GitHub API forks...")
    sd_url = clone_with_fallbacks(sd_dest,
        find_github_forks("Stability-AI", "stablediffusion"),
        "Stable Diffusion (fork)")

# Strategy 3: GitHub search
if not sd_url:
    print("\n  🔍 Trying GitHub search...")
    sd_url = clone_with_fallbacks(sd_dest,
        search_github_repos("stablediffusion"),
        "Stable Diffusion (search)")

# Validate repo structure
if sd_url:
    if os.path.isdir(os.path.join(sd_dest, "ldm", "modules", "midas")):
        print("  ✅ Repo validated (ldm.modules.midas present)")
    else:
        print("  ⚠️  WARNING: Missing ldm.modules.midas — may be an SD 1.x repo")
else:
    print("\n  🚨 CRITICAL: Could not clone stablediffusion from any source!")
    print("     Visit https://github.com/Stability-AI/stablediffusion/forks")
    print("     Find a public fork and add its URL to the list above.")

# Ensure 'taming' module is available (some forks don't bundle it)
taming_dir = os.path.join(sd_dest, "taming")
if sd_url and not os.path.isdir(taming_dir):
    print("\n  📦 Installing missing 'taming' module...")
    r = subprocess.run(
        ["git", "clone", "--config", "core.filemode=false",
         "https://github.com/CompVis/taming-transformers.git",
         "repositories/taming-transformers"],
        capture_output=True, text=True,
    )
    if r.returncode == 0:
        os.symlink(
            "/content/stable-diffusion-webui/repositories/taming-transformers/taming",
            taming_dir,
        )
        print("  ✅ taming-transformers symlinked")
    else:
        print("  ❌ Failed to clone taming-transformers")
elif sd_url:
    print("  ✅ 'taming' module present")

# Save working URL + commit hash for Cell 3
if sd_url:
    head = subprocess.run(
        ["git", "-C", sd_dest, "rev-parse", "HEAD"],
        capture_output=True, text=True,
    ).stdout.strip()
    with open(".sd_repo_info.json", "w") as f:
        json.dump({"url": sd_url, "commit": head}, f)
    print(f"  📝 Saved: {sd_url} @ {head[:12]}")

# Clone remaining repos (your forks first, originals as fallback)
!git clone https://github.com/lancedesk/generative-models.git repositories/generative-models || \
  git clone https://github.com/Stability-AI/generative-models.git repositories/generative-models
!git clone https://github.com/crowsonkb/k-diffusion.git repositories/k-diffusion
!git clone https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
!git clone https://github.com/salesforce/BLIP.git repositories/BLIP

print("\n" + "=" * 60)
print("  ✅ CELL 1 COMPLETE — Now run Cell 2")
print("=" * 60)

In [ ]:
# ============================================================
# Cell 2: Download Realistic Vision V6 (photorealistic model)
# Same SD 1.5 architecture — runs perfectly on T4 GPU (15GB VRAM)
# Runtime: ~1-2 minutes
# ============================================================

print("=" * 60)
print("  CELL 2: Downloading Realistic Vision V6 (photorealistic)")
print("=" * 60)

!mkdir -p /content/stable-diffusion-webui/models/Stable-diffusion

# Realistic Vision V6.0 B1 — the best photorealistic SD 1.5 model
# Same architecture & VRAM as base SD 1.5, but fine-tuned on real photos
print("\n📦 Downloading Realistic Vision V6 (~2 GB)...")
!wget -c https://civitai.com/api/download/models/245598 \
  -O /content/stable-diffusion-webui/models/Stable-diffusion/realisticVision_v60B1.safetensors

# Verify download
import os
model_path = "/content/stable-diffusion-webui/models/Stable-diffusion/realisticVision_v60B1.safetensors"
if os.path.exists(model_path):
    size_gb = os.path.getsize(model_path) / (1024**3)
    print(f"  ✅ Realistic Vision V6 downloaded ({size_gb:.1f} GB)")
else:
    print("  ❌ Download failed! Try re-running this cell.")

print("\n" + "=" * 60)
print("  ✅ CELL 2 COMPLETE — Now run Cell 3")
print("=" * 60)

In [ ]:
# ============================================================
# Cell 3: Launch WebUI
# Runtime: ~2-3 minutes (first launch installs remaining deps)
# ============================================================

print("=" * 60)
print("  CELL 3: Launching Stable Diffusion WebUI")
print("=" * 60)
print("\n⏳ First launch takes 2-3 min (one-time package install)...")
print("📍 A trycloudflare.com URL will appear below — open it!\n")

import sys, os, json, subprocess, time, re

os.chdir('/content/stable-diffusion-webui')

# ── Ensure numpy < 2 constraint is active ──
# ADetailer → ultralytics → numpy 2.x breaks scikit-image (np.float_ removed).
# PIP_CONSTRAINT forces ALL pip installs (including launch.py's) to respect the pin.
constraint_file = '/tmp/pip-constraints.txt'
if not os.path.exists(constraint_file):
    with open(constraint_file, 'w') as f:
        f.write('numpy<2\n')
os.environ['PIP_CONSTRAINT'] = constraint_file
print("✅ numpy < 2 constraint active")

# ── Tell launch.py to skip git operations for the SD repo ──
# Stability-AI/stablediffusion is private. launch.py's git_clone() would
# try to fetch from it. Setting the commit hash to match HEAD makes
# git_clone() return immediately (hashes match → nothing to do).
marker_file = '.sd_repo_info.json'
if os.path.exists(marker_file):
    info = json.load(open(marker_file))
    os.environ['STABLE_DIFFUSION_REPO'] = info.get('url', '')
    os.environ['STABLE_DIFFUSION_COMMIT_HASH'] = info.get('commit', '')
    print(f"✅ SD repo: {info['url']}")
    print(f"✅ Commit:  {info['commit'][:12]}...")
else:
    sd_repo = 'repositories/stable-diffusion-stability-ai'
    if os.path.isdir(os.path.join(sd_repo, '.git')):
        head = subprocess.run(['git', '-C', sd_repo, 'rev-parse', 'HEAD'],
                              capture_output=True, text=True).stdout.strip()
        os.environ['STABLE_DIFFUSION_COMMIT_HASH'] = head
        print(f"⚠️  No marker file — using HEAD: {head[:12]}")
    os.environ['STABLE_DIFFUSION_REPO'] = 'https://github.com/CompVis/stable-diffusion.git'

if not os.path.exists('launch.py'):
    print("❌ launch.py not found! Re-run Cell 1.")
    sys.exit(1)

# ── Pre-configure UI defaults (optimized for Realistic Vision V6) ──
# Writes ui-config.json so all settings are ready on first open.
print("\n⚙️  Writing ui-config.json (optimized for Realistic Vision V6)...")

ui_config = {
    # Sampler: DPM++ SDE + Karras — recommended for Realistic Vision V6
    "txt2img/Sampling method/value": "DPM++ SDE",
    "txt2img/Schedule type/value": "Karras",
    "txt2img/Sampling steps/value": 30,

    # Base resolution (3:2 aspect, SD 1.5 sweet spot)
    "txt2img/Width/value": 768,
    "txt2img/Height/value": 512,

    # Batch
    "txt2img/Batch count/value": 4,
    "txt2img/Batch size/value": 1,

    # CFG 5 — lower is more natural/photographic for Realistic Vision
    # (7+ makes images look over-saturated and "AI-looking")
    "txt2img/CFG Scale/value": 5,
    "txt2img/Seed/value": -1,

    # Hires fix: 768×512 → 1200×800
    "txt2img/Hires. fix/value": True,
    "txt2img/Upscaler/value": "R-ESRGAN 4x+",
    "txt2img/Hires steps/value": 15,
    "txt2img/Denoising strength/value": 0.40,
    "txt2img/Upscale by/value": 1.0,
    "txt2img/Resize width to/value": 1200,
    "txt2img/Resize height to/value": 800,

    # img2img (same optimized settings)
    "img2img/Sampling method/value": "DPM++ SDE",
    "img2img/Schedule type/value": "Karras",
    "img2img/Sampling steps/value": 30,
    "img2img/CFG Scale/value": 5,
    "img2img/Width/value": 1200,
    "img2img/Height/value": 800,

    # ADetailer (hand/face fixer)
    "txt2img/ADetailer/ADetailer enable/value": True,
}

ui_path = 'ui-config.json'
if os.path.exists(ui_path):
    existing = json.load(open(ui_path))
    existing.update(ui_config)
    ui_config = existing
json.dump(ui_config, open(ui_path, 'w'), indent=2)

print("  ✅ DPM++ SDE + Karras, 30 steps, CFG 5 (photorealistic)")
print("  ✅ 768×512 → Hires fix → 1200×800 (R-ESRGAN 4x+)")
print("  ✅ Batch count 4, ADetailer enabled")

# ── Cloudflare Tunnel (replaces Gradio --share) ──
print("\n📦 Setting up Cloudflare Tunnel...")
if not os.path.exists('/usr/local/bin/cloudflared'):
    subprocess.run(['wget', '-q',
        'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb',
        '-O', '/tmp/cloudflared.deb'], check=True)
    subprocess.run(['dpkg', '-i', '/tmp/cloudflared.deb'], capture_output=True)
    print("  ✅ cloudflared installed")
else:
    print("  ✅ cloudflared already installed")

print("  🔗 Starting tunnel...")
tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:7860'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
)
time.sleep(5)

tunnel_url = None
for _ in range(10):
    line = tunnel.stderr.readline()
    m = re.search(r'(https://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
    if m:
        tunnel_url = m.group(1)
        break

if tunnel_url:
    print(f"\n  {'═' * 56}")
    print(f"  🌐  PUBLIC URL: {tunnel_url}")
    print(f"  {'═' * 56}")
    print(f"  Open this URL once 'Model loaded' appears below.\n")
else:
    print("  ⚠️  URL not captured yet — look for trycloudflare.com in the logs.\n")

print("🚀 Launching WebUI...\n")

!python launch.py \
  --skip-torch-cuda-test \
  --skip-python-version-check \
  --xformers \
  --enable-insecure-extension-access \
  --api \
  --no-half-vae \
  --gradio-queue \
  --listen

---

## Usage

Once the WebUI opens:

1. **Verify model** — top-left dropdown should show `realisticVision_v60B1` (if not, select it and wait for it to load)
2. **Paste a prompt** — e.g. `RAW photo, elegant wedding invitation flatlay on mahogany desk, wax seal, calligraphy, soft natural lighting, product photography, 8k, ultra detailed, film grain`
3. **Paste a negative prompt** — e.g. `illustration, painting, drawing, cartoon, anime, 3d render, bad hands, extra fingers, deformed, blurry, low quality, watermark`
4. **Click Generate** — settings are pre-configured, 4 images will be generated
5. **Right-click → Save** your favorites

### Pro tips for photorealism (Realistic Vision V6)
- Start prompts with `RAW photo,` or `photograph,` to push the model toward realism
- Use `film grain, Fujifilm XT3` in prompts for natural photo feel
- Add `(deformed, distorted, disfigured:1.3), bad anatomy, bad hands` to negative prompts with emphasis weights
- CFG 5 is pre-set (lower = more natural, higher = more saturated/AI-looking)
- DPM++ SDE Karras is the recommended sampler for this model

### Pre-configured settings (optimized for Realistic Vision V6)

| Setting | Value |
|---------|-------|
| **Model** | **Realistic Vision V6** |
| Sampler | DPM++ SDE |
| Schedule | Karras |
| Steps | 30 |
| CFG Scale | 5 |
| Base size | 768 × 512 |
| Hires fix | ✅ → 1200 × 800 |
| Upscaler | R-ESRGAN 4x+ |
| Denoising | 0.40 |
| Batch count | 4 |
| ADetailer | ✅ (hands & faces) |

### ADetailer (fixing hands & faces)

ADetailer is pre-installed and auto-enabled. It detects hands/faces after generation and repaints them at higher detail.

- Scroll below the main settings to find the **ADetailer** section
- `hand_yolov8n.pt` — auto-fixes hands
- `face_yolov8n.pt` — auto-fixes faces (add as 2nd model if needed)
- Strengthen with negative prompt: `bad hands, extra fingers, fused fingers, too many fingers, missing fingers, malformed hands`
- Strengthen with positive prompt: `detailed hands, correct finger anatomy, five fingers`

### Generation speed (T4 GPU)

| | Time |
|---|---|
| First image (compilation) | ~30-60 sec |
| Subsequent (512×512) | ~10-15 sec |
| Subsequent (768→1200 hires) | ~30-45 sec |

---

## Optional: Download More Models

Run Cell 4 to download DreamShaper 8 (artistic all-rounder). The base SD 1.5 model and Anything V5 (anime) are also available — uncomment in Cell 4.

In [ ]:
# ============================================================
# Cell 4 (Optional): Download additional models
# All SD 1.5 models work on T4 GPU with no changes
# ============================================================

print("=" * 60)
print("  CELL 4: Optional additional models")
print("=" * 60)

# ── DreamShaper 8 — artistic, versatile all-rounder ──
print("\n📦 Downloading DreamShaper 8...")
!wget -c https://civitai.com/api/download/models/128713 \
  -O /content/stable-diffusion-webui/models/Stable-diffusion/dreamshaper_8.safetensors

# ── SD 1.5 Base — the original (weaker, included for reference) ──
# Uncomment if you want to compare against the base model:
# print("\n📦 Downloading SD 1.5 base model...")
# !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors \
#   -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.safetensors

# ── Anything V5 — anime style ──
# Uncomment if you want anime:
# print("\n📦 Downloading Anything V5...")
# !wget -c https://civitai.com/api/download/models/30163 \
#   -O /content/stable-diffusion-webui/models/Stable-diffusion/anything-v5.safetensors

print("\n" + "=" * 60)
print("  ✅ CELL 4 COMPLETE")
print("  In WebUI: top-left dropdown → select a model")
print("  Available: realisticVision_v60B1 (default), dreamshaper_8")
print("=" * 60)

---

## Troubleshooting

| Problem | Fix |
|---------|-----|
| **"fatal: could not read Username"** | Stability-AI repo is private — Cell 1 handles this with fallback URLs. If all fail, find a public fork at [Stability-AI/stablediffusion/forks](https://github.com/Stability-AI/stablediffusion/forks) and add it to the URL list. |
| **PyTorch version conflicts** | Cell 1 uninstalls old versions first. Factory reset runtime if persists. |
| **"INCOMPATIBLE PYTHON VERSION"** | Ignore — `--skip-python-version-check` handles this. |
| **"Out of memory"** | Reduce to 512×512, batch size 1, or disable Hires fix. Factory reset runtime. |
| **"Model not found"** | Re-run Cell 2. Verify file is ~4 GB. |
| **WebUI won't load** | Ensure T4 GPU is enabled. Factory reset runtime → re-run all cells. |
| **Slow first image** | Normal — first generation compiles kernels (~30-60 sec). Subsequent images are faster. |
| **Distorted hands/faces** | ADetailer should auto-fix. Check it's enabled below the main settings. Add hand terms to negative prompt. |
| **Tunnel URL not showing** | Look in the Cell 3 output logs for `trycloudflare.com`. |

### Full reset procedure
1. Runtime → **Factory reset runtime**
2. Runtime → Change runtime type → **T4 GPU** → Save
3. Run Cell 1 → Cell 2 → Cell 3

---

## API usage

The `--api` flag enables REST endpoints. Once the WebUI is running:

```python
import requests, base64, json
from PIL import Image
from io import BytesIO

url = "https://your-url.trycloudflare.com/sdapi/v1/txt2img"
payload = {
    "prompt": "mountain landscape, sunset, 8k",
    "negative_prompt": "blurry, low quality",
    "steps": 30,
    "cfg_scale": 7.5,
    "width": 768,
    "height": 512,
}
r = requests.post(url, json=payload).json()
img = Image.open(BytesIO(base64.b64decode(r["images"][0])))
img.save("output.png")
```

Full API docs available at `https://your-url.trycloudflare.com/docs`

---

## Resources

- [AUTOMATIC1111 Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki)
- [ADetailer](https://github.com/Bing-su/adetailer) — hand & face fixer
- [Civitai](https://civitai.com) — thousands of free models & LoRAs
- [r/StableDiffusion](https://reddit.com/r/StableDiffusion) — tips & community

---

**Made with ❤️ — Star the repo if this saved you time!**